# Generación de texto

En esta sección se implementará el código que convierte las salidas tensoriales del modelo GPT en texto.

## Repaso como LLm genera texto 

![Texto alternativo](./imgs/4.16.png)

En la figura se ilustra  el  proceso  paso  a  paso  mediante  el  cual  un  modelo  GPT  genera  texto  dado  un  contexto  de  entrada,  como  
"Hola,  soy",  a  nivel  general.  Con  cada  iteración,  el  contexto  de  entrada  crece,  lo  que  permite  al  modelo  generar  texto  coherente  y  
contextualizado. En  la  sexta  iteración,  el  modelo  ha  construido  una  oración  completa:  "Hola,  soy  un  modelo  listo  para  ayudar".

¿cómo  un  modelo  GPT  pasa  de  estos  tensores  de  salida  al  texto  generado?


![Texto alternativo](./imgs/4.16.png)

El  proceso  de  generación  del  próximo  token  detallado ilustra  un  solo  paso  en  el  que  el  modelo  GPT  genera  el  próximo  token  dado  su  entrada.

En  cada  paso,  el  modelo  genera  una  matriz  con  vectores  que  representan  los  posibles  tokens  siguientes.  El  vector  correspondiente  al  siguiente  token  se  extrae  y  se  convierte  en  una  distribución  de  probabilidad  mediante  la  función  softmax.  

Dentro  del  vector  que  contiene  las  puntuaciones  de  probabilidad  resultantes,  se  encuentra  el  índice  con  el  valor  más  alto,  que  se  traduce  en  el  ID  del  token.  Este  ID  se  decodifica  a  texto,  generando  el  siguiente  token  de  la  secuencia.

Finalmente,  este  token  se  añade  a  las  entradas  anteriores,  formando  una  nueva  secuencia  de  entrada  para  la  iteración  posterior.  Este  proceso  paso  a  paso  permite  al  modelo  generar  texto  secuencialmente,  construyendo  frases  y  oraciones  coherentes  a  partir  del  contexto  de  entrada  inicial.

En  la  práctica,  repetimos  este  proceso  a  lo  largo  de  muchas  iteraciones,  hasta  que  alcancemos  un  número  de  tokens  generados  especificado  por  el  usuario.

In [16]:
import torch
import torch.nn as nn
def generate_text_simple(model, idx, max_new_tokens, context_size): #idx  es  una  matriz  (lote,  n_tokens)  de  índices  en  el  contexto  actual 
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]                         #Recortar el contexto  actual  si  excede  el  tamaño  de  contexto  admitido 
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]                                 #Enfocarse solo en el último paso de tiempi de modo que pase (batch, n_token, vocab_size) a (batch, vocab_size)
        probas = torch.softmax(logits, dim=-1)                    
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)     
        idx = torch.cat((idx, idx_next), dim=1)                   #Agregar el indice muestreado a ña secuencia de ejecucion
    return idx

En  el  código  anterior,  la  función  generate_text_simple ,  utilizamos  una  función  softmax  para  convertir  los  logits  en  una  distribución  de  probabilidad,  de  la  cual  identificamos  la  posición  con  el  valor  más  alto  mediante  torch.argmax.  La  función  softmax  es  monótona,  lo  que  significa  que  conserva  el  orden  de  sus  entradas  al  transformarse  en  salidas.  Por  lo  tanto,  en  la  práctica,  el  paso  softmax  es  redundante,  ya  que  la  posición  con  la  puntuación  más  alta  en  el  tensor  de  salida  softmax  es  la  misma  posición  en  el  tensor  logit.  En  otras  palabras,  podríamos  aplicar  la  función  torch.argmax  directamente  al  tensor  logits  y  obtener  resultados  idénticos.  Sin  embargo,  codificamos  la  conversión  para  ilustrar  el  proceso  completo  de  transformación  de  logits  en  probabilidades,  lo  que  puede  aportar  mayor  intuición,  como  que  el  modelo  genera  el  siguiente  token  más  probable,  lo  que  se  conoce  como  decodificación  voraz.

![Texto alternativo](./imgs/4.18.png)

Se generan  los  ID  de  los  tokens  de  forma  iterativa.  Por  ejemplo,  en  la  iteración  1,  el  modelo  recibe  los  tokens  correspondientes  a  "Hola,  soy",  predice  el  siguiente  token  (con  ID  257,  que  es  "a")  y  lo  añade  a  la  entrada.  Este  proceso  se  repite  hasta  que  el  modelo  produce la  oración  completa  "Hola,  soy  un  modelo  listo  para  ayudar"  después  de  seis  iteraciones.

In [17]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)  #Añadir dimension de lote             
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [18]:
#poner el modelo en modo eval y deshabiliar los componentes aleatorios como el dropout
from gptModel import GPTModel
from gptConfig124M import GPT_CONFIG_124M

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)

model.eval()
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


In [19]:
#Utilizar el decode y transformar los ids en texto
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue


En el resulatdo anterior, el  modelo  generó  un  texto  incoherente,  que  no  se  parece  en  nada  al  texto  coherente  que  se  muestra  en la imagen anterior.  ¿Qué  ocurrió?  La  razón  por  la  que  el  modelo  no  puede  producir  texto  coherente  es  que  aún  no  lo  hemos  entrenado.  Hasta  ahora,  solo se ha  implementado  la  arquitectura  GPT  e  inicializado  una  instancia  del  modelo  GPT  con  pesos  aleatorios iniciales.